In [15]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException
import time
import re

In [ ]:
lista_equipos_urls = []
equipos_nombres = {}
# Configuración del navegador
service = Service(ChromeDriverManager().install())
options = Options()
options.add_argument("--start-maximized")
driver = webdriver.Chrome(service=service, options=options)
# URL
base_url = "https://www.ceroacero.es/edicion/campeonato-espanhol-2015-16/87618"
driver.get(base_url)
# Espera
wait = WebDriverWait(driver, 10)
# Clic en el botón de cookies
try:
    cookie_button = driver.find_element(By.CSS_SELECTOR, ".fc-button-label")
    cookie_button.click()
except:
    pass  # En caso de que no aparezca seguir con el código
try:
    cookie_button2 = driver.find_element(By.CSS_SELECTOR, ".ns-xhr8a-e-16")
    cookie_button2.click()
except:
    pass  # En caso de que no aparezca seguir con el código

# Buscar todos los enlaces de equipos
tabla = driver.find_element(By.CSS_SELECTOR, "table.zztable.stats.zz-datatable.dataTable.no-footer")
enlaces = tabla.find_elements(By.CSS_SELECTOR, 'a[href*="/equipo/"]')
for enlace in enlaces:
    href = enlace.get_attribute("href")
    equipo = href.split("/equipo/")[1].split("?")[0].rstrip("/")
    if equipo not in lista_equipos_urls:
        lista_equipos_urls.append(equipo)
    nombre = enlace.text
    equipos_nombres[equipo] = nombre
driver.quit()


In [12]:
equipos_nombres

{'barcelona': 'Barcelona',
 'real-madrid': 'Real Madrid',
 'atletico-de-madrid': 'Atlético de Madrid',
 'villarreal': 'Villarreal',
 'athletic': 'Athletic Bilbao',
 'celta-de-vigo': 'Celta de Vigo',
 'sevilla': 'Sevilla',
 'malaga/45': 'Málaga',
 'real-sociedad': 'Real Sociedad',
 'real-betis': 'Real Betis',
 'las-palmas/2547': 'Las Palmas',
 'valencia': 'Valencia',
 'espanyol/43': 'Espanyol',
 'eibar/5775': 'Eibar',
 'deportivo/42': 'Deportivo',
 'granada/2551': 'Granada',
 'sporting-gijon/2546': 'Sporting Gijón',
 'rayo-vallecano': 'Rayo Vallecano',
 'getafe/3753': 'Getafe',
 'levante/2570': 'Levante'}

In [13]:
plantilla_url = "https://www.ceroacero.es/equipo/{x}/unavailable?epoca_sidelines_id=145"

urls_por_equipo = {}

for equipo in lista_equipos_urls:
    if equipo == "athletic-bilbao":
        equipo = "athletic"
    url = plantilla_url.format(x=equipo)
    urls_por_equipo[equipo] = url

In [ ]:
def scrap_lesiones_equipo(equipo, lista_urls):
    df_lesiones_equipo = pd.DataFrame()

    # Configuración del navegador (una vez por equipo)
    service = Service(ChromeDriverManager().install())
    options = Options()
    options.add_argument("--start-maximized")
    driver = webdriver.Chrome(service=service, options=options)
    wait = WebDriverWait(driver, 10)

    for i, url in enumerate(lista_urls):
        dictio = {"columnas": [], "datos": []}
        driver.get(url)

        # Clic en el botón de cookies
        try:
            driver.find_element(By.CSS_SELECTOR, ".fc-button-label").click()
        except:
            pass
        try:
            driver.find_element(By.CSS_SELECTOR, ".ns-xhr8a-e-16").click()
        except:
            pass

        # Obtener la tabla
        info = driver.find_elements(By.CSS_SELECTOR, "table.zz-datatable.zztable.stats")
        for item in info:
            columnas = [th.text.strip() for th in item.find_elements(By.CSS_SELECTOR, "th")]
            dictio["columnas"].extend(columnas)

            filas = item.find_elements(By.CSS_SELECTOR, "tbody tr")
            for fila in filas:
                datos = [td.text.strip() for td in fila.find_elements(By.CSS_SELECTOR, "td")]
                dictio["datos"].append(datos)

        # Guardar DataFrame temporal
        if dictio["datos"]:
            df_temporal = pd.DataFrame(dictio["datos"], columns=dictio["columnas"])
            df_lesiones_equipo = pd.concat([df_lesiones_equipo, df_temporal], ignore_index=True)

        print(f"Datos de {equipo} desde {url}")

    driver.quit()
    return df_lesiones_equipo


In [ ]:
urls_por_equipo

In [36]:
df_lesiones_equipo = pd.DataFrame()
service = Service(ChromeDriverManager().install())
options = Options()
options.add_argument("--start-maximized")
driver = webdriver.Chrome(service=service, options=options)
wait = WebDriverWait(driver, 10)
for x, i in urls_por_equipo.items():

    dictio = {"columnas": [], "datos": []}
    driver.get(i)
    # Clic en el botón de cookies
    try:
        driver.find_element(By.CSS_SELECTOR, ".fc-button-label").click()
    except:
        pass
    try:
        driver.find_element(By.CSS_SELECTOR, ".ns-xhr8a-e-16").click()
    except:
        pass
    time.sleep(5)
    # Obtener la tabla
    info = driver.find_elements(By.CSS_SELECTOR, "table.zz-datatable.zztable.stats")
    for item in info:
        columnas = [th.text.strip() for th in item.find_elements(By.CSS_SELECTOR, "th")]
        dictio["columnas"].extend(columnas + ["href"])
        filas = item.find_elements(By.CSS_SELECTOR, "tbody tr")
        for fila in filas:
            datos = [td.text.strip() for td in fila.find_elements(By.CSS_SELECTOR, "td")]
            enlace_jugador = fila.find_element(By.CSS_SELECTOR, 'a[href*="/jugador/"]')
            href = enlace_jugador.get_attribute("href")
            datos.append(href)
            dictio["datos"].append(datos)
    # Guardar DataFrame temporal
    if dictio["datos"]:
        df_temporal = pd.DataFrame(dictio["datos"], columns=dictio["columnas"])
        df_temporal["equipo"] = x
        df_lesiones_equipo = pd.concat([df_lesiones_equipo, df_temporal], ignore_index=True)
    print(f"Datos de {x} desde {i}")
driver.quit()

Datos de barcelona desde https://www.ceroacero.es/equipo/barcelona/unavailable?epoca_sidelines_id=145
Datos de real-madrid desde https://www.ceroacero.es/equipo/real-madrid/unavailable?epoca_sidelines_id=145
Datos de atletico-de-madrid desde https://www.ceroacero.es/equipo/atletico-de-madrid/unavailable?epoca_sidelines_id=145
Datos de villarreal desde https://www.ceroacero.es/equipo/villarreal/unavailable?epoca_sidelines_id=145
Datos de athletic desde https://www.ceroacero.es/equipo/athletic/unavailable?epoca_sidelines_id=145
Datos de celta-de-vigo desde https://www.ceroacero.es/equipo/celta-de-vigo/unavailable?epoca_sidelines_id=145
Datos de sevilla desde https://www.ceroacero.es/equipo/sevilla/unavailable?epoca_sidelines_id=145
Datos de malaga/45 desde https://www.ceroacero.es/equipo/malaga/45/unavailable?epoca_sidelines_id=145
Datos de real-sociedad desde https://www.ceroacero.es/equipo/real-sociedad/unavailable?epoca_sidelines_id=145
Datos de real-betis desde https://www.ceroacero.

In [44]:
df_jugadores_href = df_lesiones_equipo[["JUGADOR", "href"]].drop_duplicates()
diccionario_jugadores = df_jugadores_href.set_index("JUGADOR")["href"].to_dict()

In [67]:
service = Service(ChromeDriverManager().install())
options = Options()
options.add_argument("--start-maximized")
driver = webdriver.Chrome(service=service, options=options)
wait = WebDriverWait(driver, 10)
dictio_jugadores = {}
for nombre, url in diccionario_jugadores.items():
    driver.get(url)
    try:
        driver.find_element(By.CSS_SELECTOR, ".fc-button-label").click()
    except:
        pass
    try:
        driver.find_element(By.CSS_SELECTOR, ".ns-xhr8a-e-16").click()
    except:
        pass
    time.sleep(2)
    tabla = driver.find_elements(By.CSS_SELECTOR, "#entity_bio.frame")
    nombre_elemento = tabla[0].find_elements(By.CSS_SELECTOR, ".bio")[0]
    jugador1 = nombre_elemento.text.split("NOMBRE")[1]
    jugador = jugador1.split("\n")[1]
    dictio_jugadores[nombre] = jugador
driver.quit()
    

In [70]:
df_lesiones_equipo["href"] = df_lesiones_equipo["JUGADOR"].map(dictio_jugadores)

In [71]:
df_lesiones_equipo

,JUGADOR,TIPO,SUB TIPO,DATA,FIM,href,equipo
0,Thomas Vermaelen,Suspension,Yellow Card Suspension,2016-06-27,2016-07-04,Thomas Vermaelen,barcelona
1,Andrés Iniesta,Suspension,Yellow Card Suspension,2016-05-23,2016-12-01,Andrés Iniesta Luján,barcelona
2,Claudio Bravo,Injury,Calf Injury,2016-05-02,2016-05-19,Claudio Andrés Bravo Muñoz,barcelona
3,Gerard Piqué,Suspension,Yellow Card Suspension,2016-04-18,2016-04-21,Gerard Piqué Bernabéu,barcelona
4,Luis Suárez,Suspension,Yellow Card Suspension,2016-04-04,2016-04-11,Luis Alberto Suárez Diaz,barcelona
...,...,...,...,...,...,...,...
369,Damián Suárez,Suspension,Red Card Suspension,2015-11-09,2015-11-24,Damián Nicolás Suárez Suárez,getafe/3753
370,Carlos Vigaray,Injury,Calf Injury,2015-11-03,2015-11-17,Carlos Martín Vigaray,getafe/3753
371,Álvaro Medrán,Injury,Broken Leg,2015-10-09,2016-02-08,Álvaro Medrán Just,getafe/3753
372,Juan Cala,Injury,Meniscus Injury,2015-09-10,2015-10-18,Juan Torres Ruiz,getafe/3753


In [72]:
df_lesiones_equipo["equipo"] = df_lesiones_equipo["equipo"].map(equipos_nombres)

In [73]:
df_lesiones_equipo

,JUGADOR,TIPO,SUB TIPO,DATA,FIM,href,equipo
0,Thomas Vermaelen,Suspension,Yellow Card Suspension,2016-06-27,2016-07-04,Thomas Vermaelen,Barcelona
1,Andrés Iniesta,Suspension,Yellow Card Suspension,2016-05-23,2016-12-01,Andrés Iniesta Luján,Barcelona
2,Claudio Bravo,Injury,Calf Injury,2016-05-02,2016-05-19,Claudio Andrés Bravo Muñoz,Barcelona
3,Gerard Piqué,Suspension,Yellow Card Suspension,2016-04-18,2016-04-21,Gerard Piqué Bernabéu,Barcelona
4,Luis Suárez,Suspension,Yellow Card Suspension,2016-04-04,2016-04-11,Luis Alberto Suárez Diaz,Barcelona
...,...,...,...,...,...,...,...
369,Damián Suárez,Suspension,Red Card Suspension,2015-11-09,2015-11-24,Damián Nicolás Suárez Suárez,Getafe
370,Carlos Vigaray,Injury,Calf Injury,2015-11-03,2015-11-17,Carlos Martín Vigaray,Getafe
371,Álvaro Medrán,Injury,Broken Leg,2015-10-09,2016-02-08,Álvaro Medrán Just,Getafe
372,Juan Cala,Injury,Meniscus Injury,2015-09-10,2015-10-18,Juan Torres Ruiz,Getafe


In [74]:
df_lesiones_equipo.to_csv("data/lesiones_completo.csv", index=False)